In [1]:
import numpy as np
from matplotlib import pyplot as plt
import math

# for circuit construction
from qiskit import QuantumCircuit, ClassicalRegister, QuantumRegister

# QFT circuit needed for phase estimation
from qiskit.circuit.library import QFT

# for creating custom gates
from qiskit import quantum_info as qi

# for simulation
from qiskit_aer import Aer
from qiskit import transpile
from qiskit.visualization import plot_histogram

# for storing data later
import pandas as pd

In [2]:
%run 'hamiltonian.ipynb' ## to get the hamiltonian cycles function

cities = ["A", "B", "C", "D", "E"]
cycles = hamiltonian_cycles(cities, symmetric = True)

In [3]:
cycles

['ABCDEA',
 'ABCEDA',
 'ABDCEA',
 'ABDECA',
 'ABECDA',
 'ABEDCA',
 'ACBDEA',
 'ACBEDA',
 'ACDBEA',
 'ACEBDA',
 'ADBCEA',
 'ADCBEA']

In [4]:
def hamiltonian_sum(cycle, edge_weights):
    return sum(edge_weights[(list(cycle)[i], list(cycle)[i+1])] for i in range(len(cycle) - 1))

In [5]:
edge_weights = {
    ('A', 'B'): 10,
    ('A', 'C'): 5,
    ('A', 'D'): 7,
    ('A', 'E'): 9,
    ('B', 'C'): 1,
    ('B', 'D'): 5,
    ('B', 'E'): 4,
    ('C', 'D'): 8,
    ('C', 'E'): 1,
    ('D', 'E'): 3,
}

# ensuring edge-weights are bi-directional
for (city1, city2), weight in list(edge_weights.items()):
    edge_weights[(city2, city1)] = weight

cycle_weights = [hamiltonian_sum(cycle, edge_weights) for cycle in cycles]
cycle_weights

[31, 22, 33, 24, 30, 30, 23, 20, 31, 22, 23, 29]

In [6]:
## weights not satisfying constraints set to zero

edge_weights_c = {
    ('A', 'B'): 0,
    ('A', 'C'): 5,
    ('A', 'D'): 7,
    ('A', 'E'): 0,
    ('B', 'C'): 1,
    ('B', 'D'): 5,
    ('B', 'E'): 4,
    ('C', 'D'): 8,
    ('C', 'E'): 1,
    ('D', 'E'): 3,
}

for (city1, city2), weight in list(edge_weights_c.items()):
    edge_weights_c[(city2, city1)] = weight
    
cycle_weights_c = [hamiltonian_sum(cycle, edge_weights_c) for cycle in cycles]
cycle_weights_c

[12, 12, 14, 14, 20, 20, 14, 20, 22, 22, 14, 20]

In [7]:
#solutions
indices = np.where(np.array(cycle_weights) == np.array(cycle_weights_c))[0]
print(indices)
np.array(cycles)[indices]

[7 9]


array(['ACBEDA', 'ACEBDA'], dtype='<U6')

In [8]:
weights = list(edge_weights.values())

#sorting edge weights
sorted_weights = np.sort(weights)[::-1]

# normalization factor
S = np.sum(sorted_weights[:4])

# epsilon
eps = 2

weights = weights / (S + eps)
weights

array([0.25 , 0.125, 0.175, 0.225, 0.025, 0.125, 0.1  , 0.2  , 0.025,
       0.075, 0.25 , 0.125, 0.175, 0.225, 0.025, 0.125, 0.1  , 0.2  ,
       0.025, 0.075])

In [9]:
for (city1, city2), weight in list(edge_weights.items()):
    edge_weights[(city1, city2)] = weight/ (S + eps)

for (city1, city2), weight in list(edge_weights_c.items()):
    edge_weights_c[(city1, city2)] = weight/ (S + eps)

cycle_weights = [hamiltonian_sum(cycle, edge_weights) for cycle in cycles]
cycle_weights_c = [hamiltonian_sum(cycle, edge_weights_c) for cycle in cycles]

sols = np.zeros((12,2))
for i, (cycle, cycle_c) in enumerate(zip(cycle_weights, cycle_weights_c)):
    sols[i][0] = np.round(cycle,4)
    sols[i][1] = np.round(cycle_c,4)

sols

array([[0.775, 0.3  ],
       [0.55 , 0.3  ],
       [0.825, 0.35 ],
       [0.6  , 0.35 ],
       [0.75 , 0.5  ],
       [0.75 , 0.5  ],
       [0.575, 0.35 ],
       [0.5  , 0.5  ],
       [0.775, 0.55 ],
       [0.55 , 0.55 ],
       [0.575, 0.35 ],
       [0.725, 0.5  ]])

In [10]:
A,  B,   C,  D,  E = [], [], [], [], []
Ac, Bc, Cc, Dc, Ec = [], [], [], [], []

# Lists to hold all the lists
U  = [A, B, C, D, E] # before constraint
Uc = [Ac, Bc, Cc, Dc, Ec] # after constraint

for i, (city,city_c) in enumerate(zip(U,Uc)):
    for j in range(5):
        if i != j:
            city.append(np.exp(1j * edge_weights[(cities[i], cities[j])] * 2 * np.pi))
            city_c.append(np.exp(1j * edge_weights_c[(cities[i], cities[j])] * 2 * np.pi))
        else:
            city.append(1)
            city_c.append(1)
            
U  = np.kron(np.kron(np.kron(np.kron(A,B),C),D),E)
Uc = np.kron(np.kron(np.kron(np.kron(Ac,Bc),Cc),Dc),Ec)


m = 12 # number of eigenstate qubits needed
filler = 2**12 - U.shape[0] #filler ones needed to have exactly 12 qubits

U = np.append(U, np.ones(filler))
Uc = np.append(Uc, np.ones(filler))



In [11]:
table = sort_indices(*map_indices(cities, symmetric = True))

# Base 10 and 2 conversions
table = np.array(table)
indices = table[:,2]
base_10 = np.array([int(indices[i], len(cities)) for i in range(len(indices))])
base_2  = np.array(["{0:b}".format(base_10[i]) for i in range(len(base_10))]) 
table = np.append(table, base_10.reshape(-1,1), axis=1)
table = np.append(table, base_2.reshape(-1,1), axis=1)
print("cycle, index, sorted_index, base 10, base 2 \n",table)

cycle, index, sorted_index, base 10, base 2 
 [['ABCDEA' '12340' '12340' '970' '1111001010']
 ['ABCEDA' '12430' '12403' '978' '1111010010']
 ['ABDCEA' '13240' '13420' '1110' '10001010110']
 ['ABDECA' '13420' '13042' '1022' '1111111110']
 ['ABECDA' '14230' '14302' '1202' '10010110010']
 ['ABEDCA' '14320' '14023' '1138' '10001110010']
 ['ACBDEA' '21340' '23140' '1670' '11010000110']
 ['ACBEDA' '21430' '24103' '1778' '11011110010']
 ['ACDBEA' '23140' '24310' '1830' '11100100110']
 ['ACEBDA' '24130' '23401' '1726' '11010111110']
 ['ADBCEA' '31240' '32410' '2230' '100010110110']
 ['ADCBEA' '32140' '34120' '2410' '100101101010']]


In [12]:
## confirming where the solutions are in matrix-U and constrained matrix-Up

table = np.array(table)
indices = table[:,2]
base_10 = np.array([int(indices[i], len(cities)) for i in range(len(indices))])
checks = np.angle(U[base_10])/2/np.pi + 1
checks_c = np.angle(Uc[base_10])/2/np.pi
checks[7] -= 1
checks_c[9] += 1
checks_c[8] += 1

check_sols = np.zeros((12,2))
for i, (check, check_c) in enumerate(zip(checks, checks_c)):
    check_sols[i][0] = np.round(check,4)
    check_sols[i][1] = np.round(check_c,4)

print(check_sols)
np.all(sols == check_sols)

[[0.775 0.3  ]
 [0.55  0.3  ]
 [0.825 0.35 ]
 [0.6   0.35 ]
 [0.75  0.5  ]
 [0.75  0.5  ]
 [0.575 0.35 ]
 [0.5   0.5  ]
 [0.775 0.55 ]
 [0.55  0.55 ]
 [0.575 0.35 ]
 [0.725 0.5  ]]


True

In [13]:
# creating gates
U = np.diag(U)
Uc = np.diag(Uc)

In [21]:
np.diag(np.matmul(np.conjugate(U), U))

array([1.+0.j, 1.+0.j, 1.+0.j, ..., 1.+0.j, 1.+0.j, 1.+0.j])

In [26]:
Ugate = qi.Operator(np.diag(U)).to_instruction()
Ugate.label = "CU"
CUgate = Ugate.control()

In [ ]:
Ucgate = qi.Operator(Uc).to_instruction()
Ucgate.label = "CU'"
CUcgate = Ucgate.control()